In [1]:
import numpy as np

In [43]:
class Func1:
    """Mapping from R^d -> R"""
    def __init__(self):
        pass
        
    def __call__(self, x):
        assert x.shape == (2, 1)
        return x[0]**2 + x[0]*x[1] + x[1]**2
    
    def grad(self, x, h):
        """Vector of first order partial derivatives of f at x"""
        out = np.zeros_like(x)
        for i in range(x.shape[0]):
            x[i] += h
            f_plus = self.__call__(x)
            x[i] -= 2*h
            f_minus = self.__call__(x)
            x[i] += h
            out[i] = (f_plus - f_minus)/(2*h)
        return out
    
    def grad2(self, x, h):
        """Matrix of second order partial derivatives of f at x"""
        out = np.zeros((x.shape[0], x.shape[0]))
        grad = self.grad(x, h)
        for i in range(x.shape[0]):
            grad[i] += h
            grad_plus = self.grad(grad, h)
            grad[i] -= 2*h
            grad_minus = self.grad(grad, h)
            grad[i] += h
            out[i] = np.squeeze(((grad_plus) - (grad_minus))/(2*h))
        return out       

In [44]:
f = Func1()
x = np.array([5, 6], dtype = np.float)
x = x[:, None]
f(x)

array([91.])

In [45]:
f.grad(x, 1e-5)

array([[16.],
       [17.]])

In [46]:
f.grad2(x, 1e-5)

array([[2.00003569, 1.00044417],
       [1.00044417, 1.99975148]])

In [47]:
def NewtonUpdate(f, x, h):
    """Expensive - O(d^2) f evals and O(d^3) for H^-1, but much faster convergence"""
    delta = np.matmul(np.linalg.inv(f.grad2(x, h)), f.grad(x, h))
    return delta

In [70]:
f = Func1()
h = 1e-4
a = 5
b = 10
iters = 5
init = (b - a)*np.random.rand(2, 1) + a
x = init
print(init, f(x))
for i in range(iters):
    x -= NewtonUpdate(f, x, h)
    print(x, f(x))

[[9.86184366]
 [9.39361726]] [278.13439052]
[[7.82329192e-06]
 [7.45172857e-06]] [1.75029203e-10]
[[ 6.77626358e-21]
 [-1.69406589e-21]] [3.73081703e-41]
[[ 2.52345232e-21]
 [-2.75726871e-21]] [7.01250623e-42]
[[0.]
 [0.]] [0.]
[[0.]
 [0.]] [0.]


In [57]:
def GradientUpdate(f, x, h, lr):
    """Less expensive - O(d) f evals, but much slower convergence"""
    delta = lr*f.grad(x, h)
    return delta

In [73]:
f = Func1()
h = 1e-4
lr = 0.01
a = 5
b = 10
iters = 600
init = (b - a)*np.random.rand(2, 1) + a
x = init
print(init, f(x))
for i in range(iters):
    x -= GradientUpdate(f, x, h, lr)
    if i % 100 == 0:
        print(x, f(x))

[[8.04514294]
 [8.68382815]] [209.99583501]
[[7.7974018 ]
 [8.42970016]] [197.58907876]
[[0.27009887]
 [0.50154052]] [0.45996183]
[[-0.02401087]
 [ 0.06070426]] [0.00280397]
[[-0.01463181]
 [ 0.01637667]] [0.00024266]
[[-0.00563357]
 [ 0.00571654]] [3.22113871e-05]
[[-0.00207528]
 [ 0.00207923]] [4.31499107e-06]
